In [2]:
import requests
import pandas as pd
import requests
from bs4 import BeautifulSoup
import json
import re

In [3]:
sitemaps = pd.read_xml("https://www.hln.be/sitemap-news.xml")
sitemaps.head()

,loc,lastmod,news,image
0,https://www.hln.be/gent/beklaagde-in-drugsdoss...,2023-08-03T12:12:11+00:00,NaN,NaN
1,https://www.hln.be/zoersel/na-36-uur-reizen-me...,2023-08-03T12:08:51+00:00,NaN,NaN
2,https://www.hln.be/pittem/terminale-annemarie-...,2023-08-03T12:07:53+00:00,NaN,NaN
3,https://www.hln.be/familie/krijg-je-er-hangbor...,2023-08-03T12:04:00+00:00,NaN,NaN
4,https://www.hln.be/lokeren/net-open-lynn-en-da...,2023-08-03T12:03:36+00:00,NaN,NaN


In [4]:
urls = sitemaps["loc"]
print(urls)

0       https://www.hln.be/gent/beklaagde-in-drugsdoss...
1       https://www.hln.be/zoersel/na-36-uur-reizen-me...
2       https://www.hln.be/pittem/terminale-annemarie-...
3       https://www.hln.be/familie/krijg-je-er-hangbor...
4       https://www.hln.be/lokeren/net-open-lynn-en-da...
                              ...                        
1278    https://www.hln.be/meer-sport/de-achillespees-...
1279    https://www.hln.be/binnenland/vab-alarmcentral...
1280    https://www.hln.be/wevelgem/spark-afterwork-en...
1281    https://www.hln.be/oudenaarde/vier-dagen-feest...
1282    https://www.hln.be/lokeren/fans-sporting-loker...
Name: loc, Length: 1283, dtype: object


In [5]:
def fetch_cookie(url):
    session = requests.Session()
    cookie = session.get(url).cookies
    return cookie


fetch_cookie(
    "https://www.hln.be/buitenland/wereldjongerendagen-in-lissabon-1-miljoen-katholieke-jongeren-verwacht-waaronder-500-vlamingen~abb18734/"
)

<RequestsCookieJar[Cookie(version=0, name='xAuthId', value='46919bd2-6497-4a5a-8958-235f885c8ab8', port=None, port_specified=False, domain='myprivacy.dpgmedia.be', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=1724760858, discard=False, comment=None, comment_url=None, rest={'HttpOnly': None, 'SameSite': 'Lax'}, rfc2109=False), Cookie(version=0, name='authId', value='46919bd2-6497-4a5a-8958-235f885c8ab8', port=None, port_specified=False, domain='myprivacy.dpgmedia.be', domain_specified=False, domain_initial_dot=False, path='/', path_specified=True, secure=True, expires=1724760858, discard=False, comment=None, comment_url=None, rest={'SameSite': 'Lax'}, rfc2109=False)]>

In [6]:
url = urls[0]
response = requests.get(url, cookies=fetch_cookie(url)).text
print(response)

<!DOCTYPE html>
<html dir="ltr" lang="nl" class="no-js  layout--mobile-animate-subnav layout--sidebar-mobile">

	<head>
		<meta charset="utf-8"/>
<meta http-equiv="x-ua-compatible" content="ie=edge"/>

<!-- meta data -->
<meta name="description" content="Een negenkoppige bende is donderdag veroordeeld in de Gentse correctionele rechtbank. Ze werden schuldig geacht aan het kweken, bezitten en verkopen van cannabis. Drie bendeleden zitten sinds enkele maanden in voorhechtenis en werden voor het vonnis vanuit de gevangenis van Dendermonde overgebracht naar de Gentse rechtbank. Vlak voordat ze de uitspraak zouden aanhoren, liep het echter mis."/>
<meta name="copyright" content="DPG Media NV"/>
<meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"/>
<meta name="apple-itunes-app" content="app-id=403025567"/>
<meta name="twitter:site" content="@HLN_BE"/><meta name="twitter:title" content="Beklaagde in drugsdossier spuwt vlak voor vonnis in gezicht van politieag

In [7]:
soup = BeautifulSoup(response, "html.parser")
print(soup)

<!DOCTYPE html>

<html class="no-js layout--mobile-animate-subnav layout--sidebar-mobile" dir="ltr" lang="nl">
<head>
<meta charset="utf-8"/>
<meta content="ie=edge" http-equiv="x-ua-compatible"/>
<!-- meta data -->
<meta content="Een negenkoppige bende is donderdag veroordeeld in de Gentse correctionele rechtbank. Ze werden schuldig geacht aan het kweken, bezitten en verkopen van cannabis. Drie bendeleden zitten sinds enkele maanden in voorhechtenis en werden voor het vonnis vanuit de gevangenis van Dendermonde overgebracht naar de Gentse rechtbank. Vlak voordat ze de uitspraak zouden aanhoren, liep het echter mis." name="description"/>
<meta content="DPG Media NV" name="copyright"/>
<meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/>
<meta content="app-id=403025567" name="apple-itunes-app"/>
<meta content="@HLN_BE" name="twitter:site"/><meta content="Beklaagde in drugsdossier spuwt vlak voor vonnis in gezicht van politieagent" name="twitter:title"/

In [8]:
def find_my_shit(url):
    title = soup.find_all("h1")
    article_title = title[0].text.strip()
    return article_title


print(find_my_shit(url))

Beklaagde in drugsdossier spuwt vlak voor vonnis in gezicht van politieagent


In [9]:
article = soup.find("p").text
print(article)


										Een negenkoppige bende is donderdag veroordeeld in de Gentse correctionele rechtbank. Ze werden schuldig geacht aan het kweken, bezitten en verkopen van cannabis. Drie bendeleden zitten sinds enkele maanden in voorhechtenis en werden voor het vonnis vanuit de gevangenis van Dendermonde overgebracht naar de Gentse rechtbank. Vlak voordat ze de uitspraak zouden aanhoren, liep het echter mis.


In [10]:
published_time = soup.find("meta", property="article:published_time")
test = published_time["content"]

regex = re.compile(r"\d\d\d\d-\d\d-\d\d")
test2 = regex.search(test)
# test2 = test.search(r"\d\d\d\d-\d\d-\d\d")
print(test)

2023-08-03T14:12:11.247+02:00
